In [6]:
import pandas as pd
import numpy as np
import nltk
from cogroo4py.cogroo import Cogroo
# nltk.download('punkt')
import re
import language_tool_python as lt
import uol_redacoes_xml as uol

In [ ]:
tool = lt.LanguageTool('pt-BR')
cogroo = Cogroo()
essays = uol.load()

### Generate Datasets Possible Erros in Essays

In [29]:
tool = lt.LanguageTool('pt-BR')
cogroo = Cogroo()
essays = uol.load()

In [84]:
def generate_essay_dataframe(essays):
    data = []
    for i, ess in enumerate(essays):
        data.append({
            'essay': i,
            'final_score': ess.final_score,
            'text': ess.text,
            'comp1': ess.criteria_scores['Competência 1'],
            'comp2': ess.criteria_scores['Competência 2'],
            'comp3': ess.criteria_scores['Competência 3'],
            'comp4': ess.criteria_scores['Competência 4'],
            'comp5': ess.criteria_scores['Competência 5'],
            'title': ess.title,
            'url': ess.url,
            'comments': ess.comments
            
        })
    essay_df = pd.DataFrame(data)
    return essay_df

In [47]:
def split_sentences(text):
    regex =  r'(?<=[.!?])\s*(?=(?:[^"]*"[^"]*")*[^"]*$)'  # Expressão regular para separar as sentenças

    sentences = re.split(regex, text)

    return sentences

In [48]:
def get_essays_corrections_tool(essays, tool):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', ''))
        for sent in sentences:
            matches = tool.check(sent)
            for mat in matches:
                data.append({
                    'essay': i,
                    'error': mat.ruleId,
                    'message': mat.message,
                    'category': mat.category,
                    'frase': sent
                })
    erros_df = pd.DataFrame(data)
    return erros_df

In [49]:
def get_essays_corrections_cogroo(essays, cogroo):
    data = []
    for i, ess in enumerate(essays):
        sentences = split_sentences(ess.text.replace('\n', '').split('.'))
        for sent in sentences:
            try:
                doc = cogroo.grammar_check(sent)
                matches = doc.mistakes
                for mat in matches:
                    data.append({
                        'essay': i,
                        'error': mat.rule_id,
                        'message':  mat.short_msg,
                        'frase': sent
                    })
            except:
                print('Missed sentence', sent)
    erros_df = pd.DataFrame(data)
    return erros_df

In [50]:
tool_df = get_essays_corrections_tool(essays, tool)
tool_df.to_csv('../data/correction_language_tool_2.csv', sep='|', index=False)
tool_df.head()

In [ ]:
cogroo_df = get_essays_corrections_cogroo(essays, cogroo)
cogroo_df.to_csv('../data/correction_cogroo_2.csv', sep='|', index=False)
cogroo_df.head()

Missed sentence  Existe varios fatores que somam o ensino, e a ausência destes são às causas do péssimo desempenho dos alunos dos demais colégios públicos


,essay,error,message,frase
0,0,space:EXTRA_BEFORE_RIGHT_PUNCT,Excesso de espaço antes de símbolo.,"Nessas épocas, as pessoas também sofriam grav..."
1,1,xml:124,O adjetivo na função de predicativo concorda c...,Então chega-se há outra questão pertinente à ...
2,1,space:EXTRA_BETWEEN_WORDS,Excesso de espaços entre as palavras.,"Inegavelmente, o ato de hackear é crime, e o ..."
3,1,xml:1,Não ocorre crase antes de palavras masculinas.,"Inegavelmente, o ato de hackear é crime, e o ..."
4,2,xml:117,Verificou-se erro de concordância entre o suje...,"O regime militar acabou, trazendo fim o instit..."


In [85]:
essays_df = generate_essay_dataframe(essays)
essays_df.to_csv('../data/essays.csv', sep='|', index=False)
essays_df.head()

,essay,final_score,text,comp1,comp2,comp3,comp4,comp5,title,url,comments
0,0,5.0,"A liberdade de expressão, assim como o direito...",1.0,1.0,1.0,1.0,1.0,Conflito entre liberdade de expressão e intimi...,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nTexto regular. ...
1,1,6.5,Os direitos fundamentais contidos na Constitui...,1.0,2.0,1.5,1.0,1.0,Entre direitos individuais e escândalos políticos,https://educacao.uol.com.br/bancoderedacoes/re...,"<comments>## Comentário geral\nTexto regular, ..."
2,2,2.5,"Ao se falar em liberdade de expressão, é impos...",0.5,0.5,0.5,0.5,0.5,"O freio na liberdade, para uma maior liberdade",https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nAlém dos muitos...
3,3,2.5,Sabe-se que o cidadão tem direito à privacidad...,1.0,1.0,0.5,0.0,0.0,Privilégio à informação,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nO texto começa ...
4,4,5.0,A liberdade de expressão e o direito a vida pr...,1.0,1.0,1.0,1.0,1.0,Limites entre Direitos,https://educacao.uol.com.br/bancoderedacoes/re...,<comments>## Comentário geral\nTexto regular. ...
